# SplitRec：在隐语拆分学习中使用 FeatureInferenceAttack
在联邦学习中，攻击者可以通过监听训练模型过程中传输的数值和梯度信息，攻击对方模型或数据，在一定程度上推理出有用信息，造成信息泄露。

本文考虑两方拆分学习中的特征推理攻击，将介绍[《Feature Inference Attacks on Model Predictions in Vertical Federated Learning》](https://arxiv.org/abs/2010.10152)中的 GRN 攻击方法在隐语中的使用。


## Feature Inference Attack with Generative Regression Network
特征推理攻击中，有标签的一方作为攻击方，推测对方的特征。在联邦模型训练之后，GRN 攻击方法通过一个生成回归网络（Generator Model）预测对方特征，并通过不断缩小预测特征在联邦模型的输出值和真实联邦模型输出值的差距，训练 Genertor Model，因而可以预测对方特征，如下图所示。

![fia0](resources/fia0.png)

其中 Generator Model 具体训练步骤如下：

1. 将攻击方特征（蓝色）和随机生成的数据（橙色）输入到 Generator Model 中，输出值作为预测的对方特征
2. 将攻击方特征和预测的对方特征输入已经训完的联邦模型中，计算 logit 输出
3. 利用步骤 2 中输出的 logit 与真实 logit（攻击方特征和对方真实特征输入联邦模型计算的 logit）计算损失
4. 对得到的损失进行反向传播，更新 Generator Model 参数

算法伪代码如下：

![fia1](resources/fia1.png)

loss 函数定义如下：

![fia2](resources/fia2.png)

## 隐语中的攻击方法实现
在隐语中攻击方法的实现是通过 callback 机制来完成。攻击算法基类 CallBack 位于 secretflow/ml/nn/sl/backend/torch/callback.py，我们在联邦模型训练的以下几个节点提供 hook，不同攻击方法可以通过将攻击算法实现在对应节点的 hook， 使攻击逻辑注入到联邦模型的训练过程中。

- on_train_begin
- on_train_end
- on_epoch_begin
- on_epoch_end
- on_batch_begin
- on_batch_end

用户如果需要实现自定义的攻击方法，需要

1. 定义 CustomAttacker 继承基类 Callback，将攻击逻辑实现到对应的 hook 函数中
2. 定义 attacker_builder 函数将构建 attacker 写到其中
3. 与普通 Split Learning 模型训练一样定义 sl_model, 并在调用 sl_model.fit() 时，将 callback_dict {party -> attacker_builder} 传入 callbacks 参数即可

其中步骤 1 可以参考隐语中已有的 FeatureInferenceAttacker/LabelInferenceAttacker，步骤 2 和 3 可参考下面 FeatureInferenceAttacker 的使用方式。

## Feature Inferece Attack 的隐语封装
我们在隐语中提供了多种攻击方法的封装。对于论文中的攻击方法，我们提供了 FeatureInferenceAttacker 封装，具体使用可以参考以下代码。

首先和一般 Split Learning 模型训练一样，我们将进行数据处理，并定义一个 SLModel。

然后定义调用 FeatureInferenceAttacker 的 attacker_builder，并在 SLModel fit 时将 attacker_builder 传入进行训练和攻击。 

## 环境设置

In [1]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()
# sf.init(['alice', 'bob'], address="local")
sf.init(['alice', 'bob'], address="local", debug_mode=True)
alice, bob = sf.PYU('alice'), sf.PYU('bob')
device_y = alice

The version of SecretFlow: 1.7.0b0


## 数据集介绍
这里我们使用 UCI Sensorless Drive Diagnosis 数据集，该数据集有 48 维特征 11 分类。

这里我们对数据进行纵向切分，攻击方持有 28 维特征和 label，被攻击方持有 20 维特征。

[数据集官网](http://archive.ics.uci.edu/dataset/325/dataset+for+sensorless+drive+diagnosis)

这里可以下载论文代码数据集： [drive_cleaned.csv](https://raw.githubusercontent.com/xinjianluo/featureinference-vfl/master/datasets/drive_cleaned.csv)

或直接使用我们提供的 demo 数据 drive_cleaned_demo.csv

## 准备数据

In [2]:
import numpy as np
from secretflow.data.ndarray import FedNdarray, PartitionWay
from secretflow.utils.simulation.datasets import _DATASETS, get_dataset


def prepare_data():
    data_path = get_dataset(_DATASETS['drive_cleaned'])
    full_data_table = np.genfromtxt(data_path, delimiter=',')
    samples = full_data_table[:, :-1].astype(np.float32)
    labels = full_data_table[:, -1].astype(np.int64)

    # permuate columns
    batch, columns = samples.shape
    permu_cols = np.random.permutation(columns)
    samples = samples[:, permu_cols]

    # normalize feature
    fea_min = samples.min(axis=0)
    fea_max = samples.max(axis=0)
    samples = (samples - fea_min) / (fea_max - fea_min)
    mean_attr = samples.mean(axis=0) # 样本均值

    # split train, test, pred
    random_selection = np.random.rand(samples.shape[0]) <= 0.6
    train_sample = samples[random_selection]
    train_label = labels[random_selection]
    sample_left = samples[~random_selection]
    label_left = labels[~random_selection]

    random_selection = np.random.rand(sample_left.shape[0]) <= 0.5
    test_sample = sample_left[random_selection]
    test_label = label_left[random_selection]
    pred_sample = sample_left[~random_selection]
    pred_label = label_left[~random_selection]

    return (
        train_sample,
        train_label,
        test_sample,
        test_label,
        pred_sample,
        pred_label,
        mean_attr,
    )

# 获取数据集
(
    train_fea,
    train_label,
    test_fea,
    test_label,
    pred_fea,
    pred_label,
    mean_attr,
) = prepare_data()

# bob 样本均值 bob是被攻击者
bob_mean = mean_attr[28:]

# train and test feature
fed_data = FedNdarray(
    partitions={
        alice: alice(lambda x: x[:, :28])(train_fea),
        bob: bob(lambda x: x[:, 28:])(train_fea),
    },
    partition_way=PartitionWay.VERTICAL,
)
test_fed_data = FedNdarray(
    partitions={
        alice: alice(lambda x: x[:, :28])(test_fea),
        bob: bob(lambda x: x[:, 28:])(test_fea),
    },
    partition_way=PartitionWay.VERTICAL,
)

# train and test label
test_data_label = device_y(lambda x: x)(test_label)
label = device_y(lambda x: x)(train_label)

## 定义 SL 模型结构

In [3]:
import torch
import torch.nn as nn
from secretflow.ml.nn.core.torch import BaseModule


class SLBaseNet(BaseModule): 
    def __init__(self):
        super(SLBaseNet, self).__init__()
        self.linear = nn.Linear(10, 10) # 怎么输入只有10个特征不是20和28吗 # 根本没推理。

    def forward(self, x):
        y = x
        return y

    def output_num(self):
        return 1


class SLFuseModel(BaseModule):
    def __init__(self, input_dim=48, output_dim=11):
        super(SLFuseModel, self).__init__()
        torch.manual_seed(1234)
        self.dense = nn.Sequential(
            nn.Linear(input_dim, 600),
            nn.ReLU(),
            nn.Linear(600, 300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, output_dim),
        )

    def forward(self, x):
        x = torch.cat(x, dim=1)
        return self.dense(x)

## 定义 SL Model

In [4]:
import torch.optim as optim
from torchmetrics import Accuracy, Precision
from secretflow.ml.nn.core.torch import metric_wrapper, optim_wrapper, TorchModel
from secretflow.ml.nn import SLModel


loss_fn = nn.CrossEntropyLoss
optim_fn = optim_wrapper(torch.optim.Adam)
base_model = TorchModel(
    model_fn=SLBaseNet,
    loss_fn=loss_fn,
    optim_fn=optim_fn,
    metrics=[
        metric_wrapper(Accuracy, task="multiclass", num_classes=11, average='micro'),
        metric_wrapper(Precision, task="multiclass", num_classes=11, average='micro'),
    ],
)

fuse_model = TorchModel(
    model_fn=SLFuseModel,
    loss_fn=loss_fn,
    optim_fn=optim_fn,
    metrics=[
        metric_wrapper(Accuracy, task="multiclass", num_classes=11, average='micro'),
        metric_wrapper(Precision, task="multiclass", num_classes=11, average='micro'),
    ],
)

base_model_dict = {
    alice: base_model,
    bob: base_model,
}

sl_model = SLModel(
    base_model_dict=base_model_dict,
    device_y=device_y,
    model_fuse=fuse_model,
    dp_strategy_dict=None,
    compressor=None,
    simulation=True,
    random_seed=1234,
    backend='torch',
    strategy='split_nn',
)

INFO:root:Create proxy actor <class 'secretflow.ml.nn.sl.backend.torch.strategy.split_nn.PYUSLTorchModel'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.sl.backend.torch.strategy.split_nn.PYUSLTorchModel'> with party bob.


## 定义 attacker_builder
### 定义 FeatureInferenceAttacker 中的 Generator Model

In [5]:
class Generator(nn.Module):
    def __init__(self, latent_dim=48, target_dim=20): # 生成的是被攻击者的数据
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 600),
            nn.LayerNorm(600),
            nn.ReLU(),
            nn.Linear(600, 200),
            nn.LayerNorm(200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.LayerNorm(100),
            nn.ReLU(),
            nn.Linear(100, target_dim),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.net(x)

### 定义 FeatureInferenceAttacker 中的 data_builder

In [6]:
from torch.utils.data import Dataset, DataLoader, TensorDataset


def data_builder(data, label, batch_size):
    def prepare_data():
        alice_data = data[:, :28]
        bob_data = data[:, 28:]

        alice_dataset = TensorDataset(torch.tensor(alice_data))
        alice_dataloader = DataLoader(
            dataset=alice_dataset,
            shuffle=False,
            batch_size=batch_size,
        )

        bob_dataset = TensorDataset(torch.tensor(bob_data))
        bob_dataloader = DataLoader(
            dataset=bob_dataset,
            shuffle=False,
            batch_size=batch_size,
        )

        dataloader_dict = {'alice': alice_dataloader, 'bob': bob_dataloader}
        return dataloader_dict, dataloader_dict # 返回的是两个一样的，因为攻击者和被攻击者的数据是一样的emm

    return prepare_data

### 定义 attacker_builder
这里 attacker_builder 是一个字典，其元素是参与方和对应的 attacker_builder_function，通常只需要填充攻击方和对应的 attacker_builder_function。

由于本文中的特征攻击算法需要对方 base 模型参数，这里我们通过被攻击方在训练结束时将 base 模型保存到磁盘，攻击方从磁盘对应路径加载模型得到对应 base 模型来实现，因而这里双方都有对应的 attacker_builder。

In [7]:
# 从这里导出攻击方法类

from secretflow.ml.nn.sl.attacks.fia_torch import (
    FeatureInferenceAttack,
    # SaveModelCallback,
)


def create_attacker_builder(
    model_save_path, bob_mean, pred_data, pred_label, batch_size, save_model_path
):
    def attacker_builder():
        victim_model_dict = {
            'bob': [SLBaseNet, model_save_path], # bob输入的是10啊？
        }
        optim_fn = optim_wrapper(optim.Adam, lr=0.0001)
        generator_model = TorchModel(
            model_fn=Generator,
            loss_fn=None,
            optim_fn=optim_fn,
            metrics=None,
        )

        data_buil = data_builder(pred_data, pred_label, batch_size)

        attacker = FeatureInferenceAttack(
            victim_model_dict=victim_model_dict,
            attack_party = alice,
            victim_party = bob,
            base_model_list=[alice,bob],
            generator_model_wrapper=generator_model,
            data_builder=data_buil,
            victim_fea_dim=[20],
            attacker_fea_dim=[28],
            enable_mean=True,
            enable_var=True,
            victim_mean_feature=bob_mean,
            save_attacker_path=save_model_path,
            load_attacker_path = None,
        )
        return attacker

    return attacker_builder


# in Algorithm 2 line 9, attacker will inference v_hat so attacker should get the whole federated model(which maybe unrealistic)
# victim bob will call this callback_builder to save base model first, then attacker alice loads this victim's model from the same path
# def create_victim_callback_builder(model_save_path):
#     def builder():
#         cb = SaveModelCallback(model_save_path)
#         return cb

#     return builder


batch_size = 64
import os
import shutil

fia_path = '/home/dengruijun/data/FinTech/Products/secretflow/drj-outs/FIA'
if os.path.exists(fia_path):
    shutil.rmtree(fia_path)
os.mkdir(fia_path)
model_save_path = fia_path + '/sl_model_victim'
generator_save_path = fia_path + '/generator'
fuse_model_save_path = fia_path+'/sl_model_fuse'

# callback_dict = {
#     alice: create_attacker_builder(
#         model_save_path,
#         bob_mean,
#         pred_fea,
#         pred_label,
#         batch_size,
#         generator_save_path,
#     ),
#     bob: create_victim_callback_builder(model_save_path),
# }

callback_dict = [create_attacker_builder(
        model_save_path,
        bob_mean,
        pred_fea,
        pred_label,
        batch_size,
        generator_save_path,
    )()]

## 开始训练和攻击

In [8]:
sl_model.fit(
    fed_data,
    label,
    validation_data=(test_fed_data, test_data_label),
    epochs=1,
    batch_size=batch_size,
    shuffle=False,
    random_seed=1234,
    dataset_builder=None,
    callbacks=callback_dict, # 暂时注释掉，callback完成后恢复 @caibei
)

INFO:root:SL Train Params: {'self': <secretflow.ml.nn.sl.sl_model.SLModel object at 0x7fc506bbb1c0>, 'x': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7fc5067bcb20>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7fc5067bc9a0>}, partition_way=<PartitionWay.VERTICAL: 'vertical'>), 'y': <secretflow.device.device.pyu.PYUObject object at 0x7fc5067bd900>, 'batch_size': 64, 'epochs': 1, 'verbose': 1, 'callbacks': [<secretflow.ml.nn.sl.attacks.fia_torch.FeatureInferenceAttack object at 0x7fc4f3156920>], 'validation_data': (FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7fc5067bd4e0>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7fc5067bd630>}, partition_way=<PartitionWay.VERTICAL: 'vertical'>), <secretflow.device.device.pyu.PYUObject object at 0x7fc5067bd7e0>), 'shuffle': False, 'sample_weight': None, 'validation_freq': 1, 'dp_spent_step_freq': None, 'd

{'train_loss': [array(7.033101e-05, dtype=float32)],
 'train_MulticlassAccuracy': [array(0.50760156, dtype=float32)],
 'train_MulticlassPrecision': [array(0.50760156, dtype=float32)],
 'val_loss': [array(4.0557843e-05, dtype=float32)],
 'val_MulticlassAccuracy': [array(0.09241766, dtype=float32)],
 'val_MulticlassPrecision': [array(0.09241766, dtype=float32)]}

In [10]:
# 保存训练好的base model？
sl_model.save_model(base_model_path=model_save_path,
                    fuse_model_path=fuse_model_save_path)

## 总结
本文通过 UCI Sensorless Drive Diagnosis 数据集上的特征攻击任务来演示了如何通过隐语来使用 FeatureInferenceAttack。

您可以：

1. 下载并拆分数据集，准备训练、攻击使用的数据
2. 定义拆分模型结构及 SL Model
3. 定义 attacker_builder，在其中定义攻击需要的 data_builder 和 FeatureInfereceAttacker
4. 调用 SL Model 进行训练攻击

您可以在自己的数据集上进行尝试，如有任何问题，可以在 github 进行训练即可。